<a href="https://colab.research.google.com/github/Toluwase/Citation-Weights-based-on-Citation-Contexts-Semantic-Similarity/blob/main/biosentvec_cit_weight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python packages needed before running the Python codes

These packages were installed on a Google colab jupyter notebook. The syntax on unix or linux may be different.

Step 1: Instal wget (not required)

In [ ]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=379211408d45db0d2b820eadccf3a40cc4f2422c33043e3de93b5d8fd6a1f09c
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


Step 2: Install fastText

In [1]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText
!pip install fastText

fatal: destination path 'fastText' already exists and is not an empty directory.


Step 3: Install SentVec
(Please note: if you install sent2vec using 
```
$!pip install sent2vec
```
then you'll get the wrong package. Please follow the instructions in the official [Github repository](https://github.com/epfml/sent2vec) to install it correctly. Alternatively, you can use the code below. I wasted many hours trying to figure out the problem during installation because of an error: ```
make: *** No targets specified and no makefile found. Stop.
```

In [2]:
!wget https://github.com/epfml/sent2vec/archive/master.zip
!unzip master.zip
!make
!sudo pip install sent2vec-master/

--2022-03-17 06:16:00--  https://github.com/epfml/sent2vec/archive/master.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/epfml/sent2vec/zip/refs/heads/master [following]
--2022-03-17 06:16:01--  https://codeload.github.com/epfml/sent2vec/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip.1’

master.zip.1            [ <=>                ] 356.24K  --.-KB/s    in 0.1s    

2022-03-17 06:16:01 (2.98 MB/s) - ‘master.zip.1’ saved [364785]

Archive:  master.zip
770bd2d475c35eccc9a2452592e7c4304ac89fb9
replace sent2vec-master/.gitignore? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sent2vec

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

# The Python script

In [6]:
import sent2vec
from nltk import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from scipy.spatial import distance

model_path = ('/content/drive/MyDrive/Colab Notebooks/BioSentVec_PubMed_MIMICIII-bigram_d700.bin')
model = sent2vec.Sent2vecModel()
try:
    model.load_model(model_path)
except Exception as e:
    print(e)
print('model successfully loaded')

stop_words = set(stopwords.words('english'))
def preprocess_sentence(text):
    text = text.replace('/', ' / ')
    text = text.replace('.-', ' .- ')
    text = text.replace('.', ' . ')
    text = text.replace('\'', ' \' ')
    text = text.lower()

    tokens = [token for token in word_tokenize(text) if token not in punctuation and token not in stop_words]

    return ' '.join(tokens)

with open("/content/drive/MyDrive/Colab Notebooks/direct_input/direct_input_100.txt") as my_file:
  lines=my_file.readlines()
#print (lines)
s1 = []
s2 = []
score = []
y=0
for everyline in lines:
    if y<len(lines)-2:
        sentenceONE=preprocess_sentence(lines[y])
        sentenceTWO=preprocess_sentence(lines[y+1])
        sentenceONE_vector = model.embed_sentence(sentenceONE)
        sentenceTWO_vector = model.embed_sentence(sentenceTWO)
        cosine_sim = 1 - distance.cosine(sentenceONE_vector, sentenceTWO_vector)
        print(lines[y],"\t", lines[y+1], "\t", 'cosine similarity:', cosine_sim)
        s1.append(lines[y])
        s2.append(lines[y+1])
        score.append(str(cosine_sim))
    y=y+3
    
#with open('output.txt', "w", encoding="utf-8") as w_f:
#    for i in range(len(s1)):
#        w_f.write(s1[i])
#        w_f.write('\t')
#        w_f.write(s2[i])
#        w_f.write('\t')
#        w_f.write(score[i])
#        w_f.write('\n')

model successfully loaded
and in 41 studies in which CBA-IF was used , 29 % were MOG-Ab-positive
 	 Summary data from 61 studies reveal the association of MOG-Abs with non-multiple sclerosis inflammatory demyelinating CNS diseases by age , and demonstrates that the proportion of individuals with an acquired demyelinating disease who are positive for MOG-Abs is greater among children ( 40 % ) than among mixed cohorts ( 29 % ) and adults ( 22 % ) . Most of these studies included selected populations , and cell-based assays ( CBA-FACS or CBA-IF ) were used to analyse MOG-Abs in children ( 18 studies )
 	 cosine similarity: 0.2460452914237976
In this review , Table 5 summarizes all of the cases of NMOSD associated with encephalitis ( n = 46 ) reported in the literature from 2010 onward
 	 While the authors of these case reports or series aimed to report clinical cases with “ overlap syndrome ” , the series by in_text_ref . was a systemic evaluation of CNS autoantibodies in pediatric demyel